In [1]:
#!/usr/bin/env python3
import cv2
import glob
import time
import imutils
import numpy as np
import mediapipe as mp
import mp_utils as mu

from matplotlib import pyplot as plt
from imutils.object_detection import non_max_suppression

In [2]:
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

##### Mediapipe Face - camera

In [4]:
width, height = 1024, 768
cap, fps = mu.setCamera(1, width, height)

debug = False
displayScale = 1
kIndex = np.array([[54, 10, 8, 162], [162, 8, 5, 93], [93, 5, 17, 58], [58, 17, 152, 150], [10, 284, 389, 8], [8, 389, 323, 5], [5, 323, 288, 17], [17, 288, 379, 152]])

cv2.namedWindow("origin", cv2.WINDOW_NORMAL)  
cv2.namedWindow("swap", cv2.WINDOW_NORMAL)  
cv2.resizeWindow("origin", (int)(width/displayScale), (int)(height/displayScale))  
cv2.resizeWindow("swap", (int)(width/displayScale), (int)(height/displayScale))
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

radius = 2
thickness = 1
color = (0, 0, 255)
frameCount = 0
type = 0

start = time.time()
delay_time = int(1000/fps)
font = cv2.FONT_HERSHEY_SIMPLEX

with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=2, 
  refine_landmarks=True, min_detection_confidence=0.5) as face_mesh:  
  while cap.isOpened():
    success, frame = cap.read()
    if not success:
      continue

    frameCount += 1
    # Flip the image horizontally for a selfie-view display.
    frame = cv2.flip(frame, 1)

    # To improve performance, optionally mark the image as not writeable to pass by reference.
    frame.flags.writeable = False
    results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    annotated_image = frame.copy()
    pannotated_image = frame.copy()
    count = 0
    if results.multi_face_landmarks:
      for face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(image=annotated_image, landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_TESSELATION, landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
        count += 1  
  
    if count == 2:
      pannotated_image = mu.kLocalPerspective(pannotated_image, kIndex, results, type, debug)

    mu.debugFrameMessage(annotated_image, font, frameCount, start, type)

    cv2.imshow("origin", annotated_image)
    cv2.imshow("swap", pannotated_image)

    key = cv2.waitKey(delay_time) & 0xFF
    if key == ord('q') or key == 27:
      break
    elif key == ord(' '):
      type += 1
      if type > 2: 
        type = 0
  cap.release()
cv2.destroyAllWindows()

fps using camera id = 1  : 30


##### Mediapipe Face - image

In [3]:
debug = True
displayScale = 1
kIndex = np.array([[21, 10, 152, 136], [10, 251, 365, 152]])
cv2.namedWindow("origin", cv2.WINDOW_NORMAL)  
cv2.namedWindow("swap", cv2.WINDOW_NORMAL)  
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
IMAGE_FILES = [".\\image\\2_faces_2.jpg", ".\\image\\2_faces.jpg"] #, ".\\image\\2_faces.jpg"

with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=2,
  refine_landmarks=True, min_detection_confidence=0.5) as face_mesh:
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    height, width, _ = image.shape
    cv2.resizeWindow("origin", (int)(width/displayScale), (int)(height/displayScale))  
    cv2.resizeWindow("swap", (int)(width/displayScale), (int)(height/displayScale))  
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    annotated_image = image.copy()
    pannotated_image = image.copy()
    if results.multi_face_landmarks:
      for face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(image=annotated_image, landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_TESSELATION, landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
  
      # pannotated_image = localPerspective(annotated_image, results)
      pannotated_image = mu.kLocalPerspective(annotated_image, kIndex, results, 2, debug)
      cv2.imshow("origin", annotated_image)
      cv2.imshow("swap", pannotated_image)
      cv2.waitKey(0)
      print(idx, file)    
  cv2.destroyAllWindows()  

0 .\image\2_faces_2.jpg
1 .\image\2_faces.jpg


#### Camera Test Mask

In [ ]:
out = mu.cameraIndexes(3)
print(out)

In [ ]:
image = cv2.imread('./image/2_faces_2.jpg')
cv2.namedWindow('origin', cv2.WINDOW_NORMAL)  
cv2.imshow('origin', image)  
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
pts = np.float32([[10,10],[80,10],[80,30],[10,30]]) #(y, x)
print(pts.shape)
print(pts[:,1], pts[:,0])

In [ ]:
image = cv2.imread(".\\image\\opencv.png")
pts1 = np.float32([[56,65],[368,52],[389,390],[28,387]])
pts2 = np.float32([[600,100],[900,100],[850,490],[600,500]])
cond = mu.bpoly2mask(pts[:,1], pts[:,0], image.shape[:2])

In [ ]:
print(image.shape)
print(cond)

In [ ]:
# mp_drawing.draw_landmarks(image=frame, landmark_list=face_landmarks,
#   connections=mp_face_mesh.FACEMESH_CONTOURS, landmark_drawing_spec=None,
#   connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
# mp_drawing.draw_landmarks(image=frame, landmark_list=face_landmarks,
#   connections=mp_face_mesh.FACEMESH_IRISES, landmark_drawing_spec=None,
#   connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style())
